<a href="https://colab.research.google.com/github/navas017/Grupo40-Practica1/blob/main/Notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1. LIBRERÍAS Y DATOS** 

### Importación de librerías

In [61]:
import pandas as pd
import numpy as np
import urllib.request
import pickle

### Importación de datos

In [62]:
# Aprendizaje
disp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/disp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

# Test
comp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/comp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

# **2. PREPARACIÓN DE DATOS**

### Matriz de Correlación

In [63]:
# Calcular la matriz de correlación
corr_matrix_disp = disp_df.corr()
corr_matrix_comp = comp_df.corr()

### Eliminación de atributos

In [64]:
# Crear listas vacía para guardar pares de atributos con correlación mayor a 0.95
high_corr_pairs_disps = []
high_corr_pairs_comp = []

# Recorrer las matrizes de correlación para disps_df y comp_df
for i in range(len(corr_matrix_disp.columns)):
    for j in range(i):
        # Verificar si el coeficiente de correlación es mayor a 0.9 y diferentes entre sí
        if abs(corr_matrix_disp.iloc[i, j]) > 0.95 and corr_matrix_disp.columns[i] != corr_matrix_disp.columns[j]:
            # Guardar pares de atributos con correlación mayor a 0.9
            high_corr_pairs_disps.append((corr_matrix_disp.columns[i], corr_matrix_disp.columns[j]))

for i in range(len(corr_matrix_comp.columns)):
    for j in range(i):
        # Verificar si el coeficiente de correlación es mayor a 0.9 y diferentes entre sí
        if abs(corr_matrix_comp.iloc[i, j]) > 0.95 and corr_matrix_comp.columns[i] != corr_matrix_comp.columns[j]:
            # Guardar pares de atributos con correlación mayor a 0.9
            high_corr_pairs_comp.append((corr_matrix_comp.columns[i], corr_matrix_comp.columns[j]))

# Crear nuevos dataframe sin los atributos correlacionados
new_disp_df = disp_df.copy()
new_comp_df = comp_df.copy()

# Eliminar los atributos correlacionados
for attr_pair in high_corr_pairs_disps:
    if attr_pair[0] in new_disp_df.columns and attr_pair[1] in new_disp_df.columns:
        # Calcular la correlación entre los atributos
        corr = comp_df[attr_pair[0]].corr(comp_df[attr_pair[1]])
        #print(f'Eliminando atributo {attr_pair[1]} debido a la alta correlación con {attr_pair[0]} (corr={corr:.2f})')
        new_disp_df.drop(attr_pair[1], axis='columns', inplace=True)

for attr_pair in high_corr_pairs_disps:
    if attr_pair[0] in new_comp_df.columns and attr_pair[1] in new_comp_df.columns:
        corr = comp_df[attr_pair[0]].corr(comp_df[attr_pair[1]])
        new_comp_df.drop(attr_pair[1], axis='columns', inplace=True)

new_corr_matrix_disp = new_disp_df.corr()

### Normalización de valores

In [65]:
# Seleccionar todas las columnas a normalizar
new_cols_to_norm_disp = new_disp_df.columns
new_cols_to_norm_comp = new_comp_df.columns

# Seleccionar las filas de entrenamiento
new_disp_first = new_disp_df.iloc[:3650, :]

# Maximos y minimos para cada columna en las filas de entrenamiento
new_dispMaxCol = new_disp_first[new_cols_to_norm_disp].max()
new_dispMinCol = new_disp_first[new_cols_to_norm_disp].min()

# Maximos y minimos para cada columna de las primeras 3650 filas para comp
new_compMaxCol = new_dispMaxCol.drop('salida', axis=0)
new_compMinCol = new_dispMinCol.drop('salida', axis=0)

# Escalar los valores de las columnas en el rango [0, 1]
new_disp_norm = (new_disp_df[new_cols_to_norm_disp] - new_dispMinCol) / (new_dispMaxCol - new_dispMinCol)
new_comp_norm = (new_comp_df[new_cols_to_norm_comp] - new_compMinCol) / (new_compMaxCol - new_compMinCol)

# Ultimas columnas para desnormalizar
new_dispMaxCol_last = new_dispMaxCol[-1]
new_dispMinCol_last = new_dispMinCol[-1]

# **3. CARGA DEL MODELO Y PREDICCIONES**

**Carga de modelo**

In [66]:
# Cargar modelo entrenado desde archivo .pkl
with open('modelo_final.pkl', 'rb') as f:
    new_model = pickle.load(f)

**Predicciones**

In [67]:
# Hacer predicciones en el conjunto de datos de prueba
preds = new_model.predict(new_comp_norm)

preds_desnorm = preds * (new_dispMaxCol_last - new_dispMinCol_last) + new_dispMinCol_last

# crea un DataFrame a partir de las predicciones desnormalizadas
df_preds = pd.DataFrame(preds_desnorm, columns=['prediccion_desnormalizada'])

# guarda el DataFrame en un archivo CSV
df_preds.to_csv('predicciones.csv', index=False)

# Configurar la impresión del array
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# Imprimir el array
print(preds_desnorm)

[10987293.    11351603.8    8246163.    11549010.    11825013.778 11594842.875 11858547.
 11752834.4   12014198.6   12958434.    10895784.    11402796.667 10991796.    13570440.
 11804388.778 12717774.    10280623.909  8104395.    10067388.437 13248837.85  11132154.176
 10469856.5   12460092.    11835654.     7125615.    12848235.    13865937.    11623287.
  7090935.     5830272.    11102916.    13186620.    13971321.    14135232.    14105466.
 12290226.    12417726.    13816959.    14164185.    14481525.    14890290.    14996154.
 15015063.    13598850.    16259475.    16671711.    16372803.    17341569.    16786826.
 16309266.     8966475.     7613361.     6129870.    10533420.     8138208.    11489067.
 12108744.    11148945.    11468241.    17694837.    15660300.    18095857.091 16930989.
  9732153.    16300314.    12652077.    16678206.    19236171.    12359484.    12417315.
 16100637.    17559429.    19812486.    20439852.    20072760.    20938014.    15475242.
 21114196.     714

# **4. CONCLUSIONES**